# Movie Recommender

The following program inputs a list of movies and generates a list of recommended movies
according to the TasteDive API and sorts them based on their Rotten Tomatoes ratings, retrieved from the OMDB API.

## Program Methodology

The ultimate goal of this application is to retrieve a list of similar movies (sorted by rating) to a provided list of movies.
This task is performed with the following steps:  
1. For each movie title in a provided list of movie titles, get a 5 similar movies from TasteDive API
2. Extract the movie titles from each TasteDive output
3. Compile all movie titles into a single list, omitting duplicate titles
4. For each movie title extracted from TasteDive, retrieve detailed data from OMDB API
5. Extract Rotten Tomatoes ratings for each
6. Sort the movie list by rating (descending order)

API Platforms: 
* https://tastedive.com/read/api
* https://www.omdbapi.com/  

In [17]:
import requests
import json

In [16]:
# API Platform: https://tastedive.com/read/api

def get_movies_from_tastedive(s, tastedive_api_key=''):
    '''
    Retrieves 5 similar movies from TasteDive API platform
    Inputs: Movie title as string, api_key as string
    Returns: dictionary of 5 similar movies
    '''
    parameters = {'k': tastedive_api_key, 'q': s, 'type': 'movies', 'limit': 5}
    response = requests.get("https://tastedive.com/api/similar", params = parameters)
    
    return json.loads(response.text)

def extract_movie_titles(d): 
    '''
    Extracts movie titles of similar movies from dictionary of movie data provided
    after calling get_movies_from_tastedive(...)
    Input: dictionary of movie data
    Returns: list of movie titles
    '''
    return [x['Name'] for x in d['Similar']['Results']]

def get_related_titles(lst, tastedive_api_key):
    '''
    Creates a list of movies similar to provided list of movies, omitting duplicates
    Inputs: list of movies, api_key as string
    Returns: list of unique similar movies
    '''
    related = []
    related_distinct = []
    for x in lst:
        related.append(extract_movie_titles(get_movies_from_tastedive(x, tastedive_api_key)))
    
    # omit duplicate movies
    for lst in related:
        for i in lst:
            if i not in related_distinct: related_distinct.append(i)
    
    return related_distinct

In [18]:
# API Platform: https://www.omdbapi.com/

def get_movie_data(s, omdb_api_key):
    '''
    Retrieves data about a movie from the OMDB API platform
    Inputs: Movie title as string, API Key as string
    Returns: Movie data in dictionary format
    '''
    parameters = {'apikey': omdb_api_key, 't': s, 'r': 'json'}
    response = requests.get("http://www.omdbapi.com/", params = parameters)
    
    return json.loads(response.text)

def get_movie_rating(d):
    '''
    Extract Rotten Tomatoes movie rating from dictionary provided by
    OMDB API
    Inputs: dictionary provided by OMDB API after calling get_movie_data(...)
    Returns: Movie Rating as int    
    '''
    for r in d['Ratings']:
        if r['Source'] == 'Rotten Tomatoes':
            return int(r['Value'].replace('%',''))
    return 0

def get_sorted_recommendations(lst, tastedive_api_key, omdb_api_key):
    '''
    Takes a list of movie titles and finds 5 related movie titles for each and 
    sorts all from highest rated to lowest rated (according to Rotten Tomatoes)
    Inputs: list of movie titles, tastedive_api_key as str, omdb_api_key as str
    Output: list of related movie titles, sorted from highest rated to lowest
    '''
    # get related movies
    related_lst = get_related_titles(lst, tastedive_api_key)
    
    # get movie ratings
    d = []
    for l in related_lst:
        d.append((l, get_movie_rating(get_movie_data(l, omdb_api_key))))
    
    # sort
    # d.sort(key = lambda x: x[0], reverse = True) Sort in reverse alphabetical order to break ties
    d.sort(key = lambda x: x[1], reverse = True)
    
    return [x[0] for x in d]

In [20]:
TASTEDIVE_API_KEY = '335161-chris-HP8EYMN8'
OMDB_API_KEY = '17cdc959'

get_sorted_recommendations(['Black Panther', 'Captain Marvel'], TASTEDIVE_API_KEY, OMDB_API_KEY)

['Ant-Man And The Wasp',
 'Ralph Breaks The Internet',
 'Avengers: Infinity War',
 'Captain Marvel',
 'Jumanji: Welcome To The Jungle',
 'The Fate Of The Furious',
 'Alita: Battle Angel',
 'The Mummy',
 'Fantastic Beasts: The Crimes Of Grindelwald',
 'Mortal Engines']